<a href="https://colab.research.google.com/github/Samgomes2510/Analise_credito_Churn/blob/main/Projeto_4_pontuacao_credito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plotly
!pip install seaborn
!pip install matplotlib
!pip install plotly.express
!pip install pandas


In [ ]:
# Importando bibliotecas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Carregando os dados
df = pd.read_csv('CREDIT_SCORE_PROJETO_PARTE1.csv', delimiter=';')

# Visualizando as primeiras linhas
df.head()


In [ ]:
# Corrigindo o formato de renda: de string para float
df['Income'] = df['Income'].str.replace('.', '').str.replace(',', '.').astype(float)

# Verificando os tipos de dados
df.dtypes

In [ ]:
# Verificando valores nulos
df.isnull().sum()

# Tratando a coluna Age preenchendo pela média (idade é numérica contínua)
df['Age'].fillna(df['Age'].mean(), inplace=True)

In [ ]:
# Verificando valores únicos
for col in df.select_dtypes(include='object'):
    print(f"\n{col}:")
    print(df[col].value_counts())

# Padronizando gênero (caso haja variação de maiúsculas/minúsculas)
df['Gender'] = df['Gender'].str.capitalize()

#**ETAPA 2 – ANÁLISE UNIVARIADA**

In [ ]:
df.describe()

In [ ]:
sns.histplot(df['Age'], kde=True)
plt.title('Distribuição da Idade')
plt.show()

In [ ]:
sns.boxplot(x=df['Income'])
plt.title('Bocplot da Renda')
plt.show()

In [ ]:
sns.countplot(data=df, x='Credit Score')
plt.title('Distribuição do Score de Crédito')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=df, x='Education', order=df['Education'].value_counts().index)
plt.title('Distribuição por Escolaridade')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Verificando outliers em renda
q1 = df['Income'].quantile(0.25)
q3 = df['Income'].quantile(0.75)
iqr = q3 - q1
limite_superior = q3 + 1.5 * iqr

# Mantendo outliers por se tratar de renda alta realista
df[df['Income'] > limite_superior]

 # **ETAPA 3 – ANÁLISE BIVARIADA**

In [ ]:
sns.boxplot(x='Marital Status', y='Age', data=df)
plt.title('Idade Por Estado Civil')
plt.show()

In [ ]:
sns.countplot(data=df, x='Education', hue='Credit Score')
plt.title('Score por Escolaridade')
plt.xticks(rotation=45)
plt.show()

In [ ]:
sns.scatterplot(data=df, x='Age', y='Income')
plt.title('Idade x Renda')
plt.show()

In [ ]:
sns.boxplot(x='Credit Score', y='Income', data=df)
plt.title('Renda Por Score de Crédito')
plt.show()

In [ ]:
sns.countplot(data=df, x='Home Ownership', hue='Credit Score')
plt.title('Score de Crédito por Tipo de Residência')
plt.show()

In [ ]:
# Score vs Gênero
sns.countplot(data=df, x='Gender', hue='Credit Score')
plt.title('Score por Gênero')
plt.show()

# Score vs Número de filhos
sns.boxplot(x='Credit Score', y='Number of Children', data=df)
plt.title('Número de Filhos por Score')
plt.show()

# Renda média por nível de escolaridade
renda_edu = df.groupby('Education')['Income'].mean().sort_values()
renda_edu.plot(kind='bar', title='Renda média por Escolaridade', figsize=(10,5))
plt.ylabel('Renda Média')
plt.show()

In [ ]:
!pip install imblearn

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE


In [ ]:
# Carregando a base de dados
df = pd.read_csv('CREDIT_SCORE_PROJETO_PARTE1.csv', delimiter=';')

df.head(10)

In [ ]:
# Tratamento da coluna 'Income'
df['Income'] = df['Income'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)



In [ ]:
# Tratando a coluna Age preenchendo pela média (idade é numérica contínua) - Repetindo para garantir no novo df
df['Age'].fillna(df['Age'].mean(), inplace=True)

In [ ]:
# Correlação entre variáveis numéricas
corr_numeric = df[['Age', 'Income', 'Number of Children']].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_numeric, annot=True, cmap='coolwarm')
plt.title('Correlação entre variáveis numéricas')
plt.show()

In [ ]:
# Codificação de variáveis categóricas com LabelEncoder
df_encoded = df.copy()
label = LabelEncoder()
for col in df_encoded.select_dtypes(include='object'):
    df_encoded[col] = label.fit_transform(df_encoded[col])


In [ ]:
# Verificando e imputando quaisquer NaNs restantes antes de separar X e y
print("Verificando NaNs antes da divisão:")
print(df_encoded.isnull().sum())

In [ ]:
# Se houver NaNs, imputar com a média (ou outra estratégia apropriada)
# Neste caso, vamos imputar com a média para colunas numéricas restantes
for col in df_encoded.select_dtypes(include='number'):
    if df_encoded[col].isnull().any():
        print(f"Imputando coluna '{col}' com a média.")
        df_encoded[col].fillna(df_encoded[col].mean(), inplace=True)

print("\nVerificando NaNs após imputação:")
print(df_encoded.isnull().sum())


In [ ]:
# Nova matriz de correlação
plt.figure(figsize=(10, 8))
sns.heatmap(df_encoded.corr(), annot=True, cmap='Blues')
plt.title('Correlação após codificação das variáveis')
plt.show()


In [ ]:
# Separando features e alvo
X = df_encoded.drop('Credit Score', axis=1)
y = df_encoded['Credit Score']


In [ ]:
# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Verificando balanceamento do alvo
sns.countplot(x=y_train)
plt.title("Distribuição do Credit Score na base de treino")
plt.show()

In [ ]:
# Aplicando SMOTE para balancear os dados de treino
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)


In [ ]:
# Verificando o balanceamento após SMOTE
sns.countplot(x=y_train_bal)
plt.title("Distribuição após SMOTE (base de treino)")
plt.show()